Rerfence: https://github.com/ruohoruotsi/LSTM-Music-Genre-Classification/blob/master/lstm_genre_classifier_keras.py
                  https://gist.github.com/parulnith/7f8c174e6ac099e86f0495d3d9a4c01e

In [0]:
!wget http://opihi.cs.uvic.ca/sound/genres.tar.gz

--2019-04-18 04:12:42--  http://opihi.cs.uvic.ca/sound/genres.tar.gz
Resolving opihi.cs.uvic.ca (opihi.cs.uvic.ca)... 142.104.68.135
Connecting to opihi.cs.uvic.ca (opihi.cs.uvic.ca)|142.104.68.135|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1225573614 (1.1G) [application/x-gzip]
Saving to: ‘genres.tar.gz’

genres.tar.gz       100%[===================>]   1.14G  10.7MB/s    in 98s     

2019-04-18 04:14:20 (11.9 MB/s) - ‘genres.tar.gz’ saved [1225573614/1225573614]



In [0]:
!tar xvzf genres.tar.gz

genres/
genres/blues/
genres/blues/blues.00000.au
genres/blues/blues.00001.au
genres/blues/blues.00002.au
genres/blues/blues.00003.au
genres/blues/blues.00004.au
genres/blues/blues.00005.au
genres/blues/blues.00006.au
genres/blues/blues.00007.au
genres/blues/blues.00008.au
genres/blues/blues.00009.au
genres/blues/blues.00010.au
genres/blues/blues.00011.au
genres/blues/blues.00012.au
genres/blues/blues.00013.au
genres/blues/blues.00014.au
genres/blues/blues.00015.au
genres/blues/blues.00016.au
genres/blues/blues.00017.au
genres/blues/blues.00018.au
genres/blues/blues.00019.au
genres/blues/blues.00020.au
genres/blues/blues.00021.au
genres/blues/blues.00022.au
genres/blues/blues.00023.au
genres/blues/blues.00024.au
genres/blues/blues.00025.au
genres/blues/blues.00026.au
genres/blues/blues.00027.au
genres/blues/blues.00028.au
genres/blues/blues.00029.au
genres/blues/blues.00030.au
genres/blues/blues.00031.au
genres/blues/blues.00032.au
genres/blues/blues.00033.au
genres/blues/blues.00034.a

In [0]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
import pathlib
import csv
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import keras
from keras import models
from keras import layers

import warnings
warnings.filterwarnings('ignore')

from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam

import timeit


Using TensorFlow backend.


In [0]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./genres/{g}'):
        songname = f'./genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        #plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 720x720 with 0 Axes>

In [0]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate flatness poly chroma_cqt chroma_cens spectral_contrast' #melspectrogram
for i in range(1, 21):
    header += f' mfcc{i}' 
for i in range(1, 7):
    header += f' tonnetz{i}'    
header += ' label'
header = header.split()

In [0]:
file = open('songs.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()

In [0]:
start = timeit.default_timer()

rmse = librosa.feature.rmse(y=y)

for g in genres:
    for filename in os.listdir(f'./genres/{g}'):
        songname = f'./genres/{g}/{filename}'
        
        # new features
        flatness = librosa.feature.spectral_flatness(y=y)
        
        S = np.abs(librosa.stft(y))
        p0 = librosa.feature.poly_features(S=S, order=0)
        
        tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
        
        chroma_cq = librosa.feature.chroma_cqt(y=y, sr=sr)
        
        chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr) 
        
        S = np.abs(librosa.stft(y))
        contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
        
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
                
        
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)} {np.mean(flatness)} {np.mean(p0)} {np.mean(chroma_cq)} {np.mean(chroma_cens)} {np.mean(contrast)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'

        for i in tonnetz:
            to_append += f' {np.mean(i)}'
        to_append += f' {g}'
        file = open('songs.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())
            
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  2883.6949392469996


In [0]:
ls

genres/  genres.tar.gz  img_data/  sample_data/  songs.csv


In [0]:
from google.colab import files
files.download('songs.csv') 

In [0]:
data = pd.read_csv('songs.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,flatness,poly,chroma_cqt,...,mfcc18,mfcc19,mfcc20,tonnetz1,tonnetz2,tonnetz3,tonnetz4,tonnetz5,tonnetz6,label
0,blues.00019.au,0.257259,0.09198,1195.411640,1481.284259,2234.878153,0.058871,0.017725,1.086251,0.555419,...,-1.337586,-3.719717,-4.353154,0.020575,0.016943,0.088439,-0.026695,0.012183,-0.005494,blues
1,blues.00053.au,0.412501,0.09198,2171.221742,1954.383785,4237.132712,0.132802,0.000642,0.594293,0.513369,...,-0.271427,-3.696092,-0.504041,-0.002745,0.008571,-0.042509,0.057754,0.005468,0.017238,blues
2,blues.00048.au,0.379428,0.09198,2148.786199,2204.574064,4673.663406,0.098812,0.005946,1.750953,0.594625,...,0.464257,-1.909089,-3.128308,-0.010565,0.000952,0.003099,0.026811,-0.004230,0.006860,blues
3,blues.00001.au,0.340983,0.09198,1529.835316,2038.617579,3548.820207,0.056044,0.005735,1.223702,0.509443,...,0.293875,-0.287431,0.531573,-0.020591,0.000802,0.031767,-0.016651,0.005038,0.005216,blues
4,blues.00058.au,0.356588,0.09198,2068.371125,2033.930047,4231.499413,0.104372,0.002294,0.675315,0.503215,...,0.194597,0.842872,-3.416576,0.007437,0.018288,0.046516,0.011819,0.007859,-0.007299,blues
